# Purcell filter scattering

This notebook creates a generalized version of Fig. 4b in [Guillermo F. Peñas et al (2021)](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.17.054038). In that work, Fig. 4b describes the phase shift experienced by a single photon in a quantum link, when scattered by the terminating cavity at the end of the link.

In this notebook, we generalize the setup, considering that said cavity cavity that is separated from the quantum link by a Purcell filter. The presence of the filter and its coupling with the cavity creates two effective resonances that can be reproduced by a simulation of the scattering process.

The notebook illustrates the following aspects from the library:

1. Creation of a model class that describes the ETH setup with two cavities, two Purcell filters and two qubits.
2. Creation of initial conditions for the simulation, with a single, broadband photon, and unexcited qubits.
3. Trivial evolution of the whole system with an efficient solver when the controls are time-independent.
4. Extraction of information from the single photon wavepacket.

## Setup

While the SuperQuLAN library can be installed, this notebook only assumes that it resides inside the `examples` folder and that the library is located under `../superqulan`.

In [1]:
import sys
if "../" not in sys.path:
    sys.path = ["../"] + sys.path

In [2]:
import numpy as np 
from numpy import floor, sqrt
import scipy.sparse.linalg
from math import pi as π
import scipy.sparse as sp
from superqulan.waveguide import Waveguide
from superqulan.architecture import Setup, Exp_2qubits_2cavities_2purcells

We configure Matplotlib to not show figures. We will inline them to avoid generating too much noise in GitHub.

In [3]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('agg')
plt.rc('font', size=20)

## Uniform photon

We perform the spectroscopy of photon-qubit interaction by creating a uniform wavepacket that contains all the $N_k$ frequencies that we want to model
$$
|\psi\rangle = \sum_{n=0}^{N_k-1} \frac{1}{\sqrt{N_k}} a^\dagger_{k_n}|\text{vac}\rangle
$$
This is achieved using the `Setup` class' tools for constructing wavefunctions.

In [4]:
def step_photon(setup: Setup) -> np.ndarray: 
    """Creates a flat photon, that is, an equiprobable distribution of all the modes 
    of the waveguide.

    Args:
        mmax (int): number of modes in the WG.

    Returns:
        photon (np.ndarray)
    """
    N = setup.waveguide.modes
    wavefunction = np.ones(N) / np.sqrt(N)
    return setup.waveguide_photon(wavefunction)

## Scattering experiment

The simulation is relatively straightforward: we create a photon in a realistic setup, which is studied in the single-excitation limit. We create a setup that consists of two uncoupled qubits, and only one cavity coupled to the waveguide via a Purcell filter.

We evolve the initial wavefunction with the broad photon for a total time equal to the propagation time of the waveguide. Note that since the collision is energy conserving, the amplitudes of the different photons are more or less conserved (the Purcell filter and cavity had time to relax) and the only effect is a phase shift that we may compute.

In [5]:
def flat_photon_scattering(
    ω_cavity=2 * π * 8.40701933547913,
    ω_filter=2 * π * 8.40701933547913,
    g_p=2 * π * 25e-3,
    κ=2 * π * 130e-3,
    length=30,
    modes=2101,
):
    """computes the scattering of a flat photon that bounces off two
    coupled harmonic oscillators..

    Args:
        ω_cavity (double):  frequency of the cavity
        ω_filter (double):  frequency of the filter
        g_p (double):       cavity-filter coupling
        κ (double):         decay-rate of the filter
        length (double):    length of the waveguide
        modes (int):        number of modes for the waveguide simulation

    Returns:
        frequencies:        waveguide frequencies relative to the cavity
        phases:             phase shift experienced by each mode (in units of π rads)
        phases_theory:      crude theory prediction for the phase shift
    """

    # We create a setup with given parameters:
    setup = Exp_2qubits_2cavities_2purcells(
        # two identical cavities
        ω1=ω_cavity,  
        ω2=ω_cavity,
        # two identical filters
        ωp1=ω_filter,  
        ωp2=ω_filter,
        # two disconnected qubits that do not participate in the dynamics
        g1=0,  
        g2=0,
        # only the second cavity is coupled to the filter
        gp1=0,  
        gp2=g_p,
        # only the second filter is coupled to the waveguide
        κ1=0,  
        κ2=κ,
        # The waveguide has a fixed length, but we only study a
        # fixed number of modes around the cavity frequency
        waveguide=Waveguide(frequency=ω_cavity, modes=modes, length=length),
    )
    frequencies = setup.waveguide.frequencies

    duration = 2 * setup.waveguide.tprop

    v0 = step_photon(setup)

    vt = scipy.sparse.linalg.expm_multiply(-1j * duration * setup.Hamiltonian(), v0)
    vt = vt[setup.waveguide_indices]
    v0 = v0[setup.waveguide_indices]

    phases = np.angle(v0 * vt * np.exp(1j * frequencies * duration)) / π

    ω_difference = frequencies - ω_cavity
    phases_theory = np.real(
        -1j
        * np.log(
            ((1j * (ω_difference + g_p) + (κ / 4)))
            / (1j * (ω_difference + g_p) - (κ / 4))
            * ((1j * (ω_difference - g_p) + (κ / 4)))
            / (1j * (ω_difference - g_p) - (κ / 4))
        )
        / π
    )
    return ω_difference, phases, phases_theory


## Sample plot

In [6]:
κ1 = 2 * π * 130e-3
g_p = 2 * π * 25e-3

ω_difference, phases, phases_theory = flat_photon_scattering(g_p=g_p, modes=500)


In [7]:
fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(13, 6), constrained_layout=True)
ax1.scatter(ω_difference/κ1, phases, s=20, color="tab:blue", label="Simulation")
ax1.plot(ω_difference/κ1, phases_theory, color="tab:orange", label="theory")
ax1.set_xlabel(r"$(\omega - \Omega_{R_2})/\kappa$")
ax1.set_ylabel(r"$\phi/\pi$")
ax1.set_xlim([-3,3])
ax1.legend()
fig.savefig('figures/fig-01-001.svg');


![Fig. 1](./figures/fig-01-001.svg)